In [1]:
import os
import sys

current_dir = os.getcwd()
PROJECT_DIR = os.path.abspath(os.path.join(current_dir, '..'))
if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)
BASE_DIR = os.path.basename(PROJECT_DIR)

print(f"Project path (Relative): {BASE_DIR}")

Project path (Relative): foundation-moirai-time-series-forecasting


In [2]:
import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Device using: {DEVICE}")

Device using: cuda


In [3]:
import yaml

CONFIGS_DIR = f"{PROJECT_DIR}/configs/config.yaml"
with open(CONFIGS_DIR, "r") as f:
    config = yaml.safe_load(f)

SPY_DATASET, AQI_DATASET = config['datasets']

In [4]:
HORIZON = config['experiment']['horizon']
CONTEXT_LENGTH = config['experiment']['context_length']

print(f"Horizon: {HORIZON}")
print(f"Context length: {CONTEXT_LENGTH}")

Horizon: 60
Context length: 60


In [5]:
SEASONALITY = config['datasets'][SPY_DATASET]['seasonality']
FREQ = config['datasets'][SPY_DATASET].get('frequency', 'D')

print(f"- Seasonality: {SEASONALITY}")
print(f"-   Frequency: {FREQ}")

- Seasonality: 5
-   Frequency: B


In [6]:
def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

In [7]:
from src.moirai_wrapper import MoiraiPredictor
from src.baselines import BaselineModels

In [8]:
MOIRAI_SMALL = config['models']['moirai_small']
MOIRAI_BASE = config['models']['moirai_base']

In [9]:
baselines = BaselineModels(
    seasonal_period=SEASONALITY,
    horizon=HORIZON)

moirai_models = {
    'Moirai-Small': MoiraiPredictor(
        model_name=MOIRAI_SMALL,
        context_length=CONTEXT_LENGTH,
        horizon=HORIZON,
        device=DEVICE,
        freq=FREQ),
    
    'Moirai-Base': MoiraiPredictor(
        model_name=MOIRAI_BASE,
        context_length=CONTEXT_LENGTH,
        horizon=HORIZON,
        device=DEVICE,
        freq=FREQ)
}

In [10]:
moirai_base = moirai_models["Moirai-Base"]
module_base = moirai_base.model.module

total, trainable = count_parameters(module_base)
print(f"{total:,.0f}")
print(f"{trainable:,.0f}")

91,357,728
91,357,728


In [11]:
moirai_small = moirai_models["Moirai-Small"]
module_small = moirai_small.model.module

total, trainable = count_parameters(module_small)
print(f"{total:,.0f}")
print(f"{trainable:,.0f}")

13,827,528
13,827,528


In [13]:
for name, module_ in module_base.named_children():
    print(name, "→", module_.__class__.__name__)

mask_encoding → Embedding
scaler → PackedStdScaler
in_proj → MultiInSizeLinear
encoder → TransformerEncoder
param_proj → DistrParamProj


In [15]:
for name, module_ in module_small.named_children():
    print(name, "→", module_.__class__.__name__)

mask_encoding → Embedding
scaler → PackedStdScaler
in_proj → MultiInSizeLinear
encoder → TransformerEncoder
param_proj → DistrParamProj


In [16]:
from collections import defaultdict

layer_params = defaultdict(int)

for name, param in module_small.named_parameters():
    layer_name = name.split('.')[0]
    layer_params[layer_name] += param.numel()

for layer, count in layer_params.items():
    print(f"{layer:20s} {count:>12,}")

mask_encoding                 384
in_proj                   247,680
encoder                10,622,664
param_proj              2,956,800


In [17]:
for name, param in module_base.named_parameters():
    layer_name = name.split('.')[0]
    layer_params[layer_name] += param.numel()

for layer, count in layer_params.items():
    print(f"{layer:20s} {count:>12,}")

mask_encoding               1,152
in_proj                   743,040
encoder                95,578,344
param_proj              8,862,720
